# FPL Points Predictor and Team Optimizer 

##### Importing necessary libraries


In [1]:
import pandas as pd
import re
import math
import requests
import csv
from unidecode import unidecode
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, LpStatus

#### Importing Data for Relevant Gameweeks

In [2]:
def parse_fpl_data_from_file(filepath: str) -> pd.DataFrame:
    """
    Parses an FPL player data file with the new 3-line format.
    - Line 1: Player Name <tab> Team Name badge
    - Line 2: Predicted Points
    - Line 3: Other stats including the Price

    Args:
        filepath: The path to the .csv or .txt file containing the raw data.

    Returns:
        A pandas DataFrame with ['Name', 'Team', 'Predicted Points', 'Price'],
        or None if parsing fails.
    """
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            lines = [line for line in f.read().strip().split('\n') if line.strip()]
    except FileNotFoundError:
        print(f"Error: The file '{filepath}' was not found.")
        return None

    if not lines:
        print("The file is empty or contains no valid data.")
        return None

    player_records = []
    
    # Iterate through the lines three at a time
    for i in range(0, len(lines), 3):
        # Ensure we have a complete 3-line block for a player
        if i + 2 < len(lines):
            # --- Line 1: Parse Name and Team ---
            line1_parts = lines[i].strip().split('\t')
            if len(line1_parts) < 2:
                continue # Skip if the line is not in the expected format

            player_name = line1_parts[0].strip()
            # Clean the team name by removing " badge"
            team_name = line1_parts[1].replace(' badge', '').strip()

            # --- Line 2: Get Predicted Points ---
            predicted_points = lines[i+1].strip()

            # --- Line 3: Find and Extract the Price ---
            stats_line = lines[i+2].strip()
            # Regex is the most robust way to find the price pattern
            price_match = re.search(r'£(\d+\.\d+)m', stats_line)

            # Only add the record if a price was successfully found
            if price_match:
                price = price_match.group(1)
                
                player_records.append({
                    'Name': player_name,
                    'Team': team_name,
                    'Predicted Points': predicted_points,
                    'Price': price
                })

    if not player_records:
        print("Could not parse any player data. Check the file format.")
        return None
        
    # --- Create and Clean the DataFrame ---
    df = pd.DataFrame(player_records)
    
    # Convert data types to numeric for calculations
    df['Predicted Points'] = pd.to_numeric(df['Predicted Points'], errors='coerce')
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
    
    # Drop rows where essential data could not be converted
    df.dropna(inplace=True)
    
    print(f"Successfully parsed {len(df)} players.")
    return df

In [3]:
def add_player_positions(gameweek_df: pd.DataFrame, positions_filepath: str) -> pd.DataFrame:
    """
    Adds a 'Position' column to a gameweek DataFrame by merging it with a
    positions CSV file.

    Args:
        gameweek_df: The DataFrame containing weekly player data (Name, Price, etc.).
        positions_filepath: The path to the 'player_positions.csv' file.

    Returns:
        A new DataFrame with the 'Position' column added. Players not found in the
        positions file will have their position as NaN and will be dropped.
    """
    try:
        positions_df = pd.read_csv(positions_filepath)
    except FileNotFoundError:
        print(f"Error: Positions file not found at '{positions_filepath}'")
        return None

    # Merge the gameweek data with the positions data using the 'Name' column
    # 'how=left' ensures all players from the original gameweek_df are kept
    merged_df = pd.merge(gameweek_df, positions_df, on='Name', how='left')

    # Check for players that didn't get a position matched
    missing_positions = merged_df[merged_df['Position'].isnull()]
    if not missing_positions.empty:
        print(f"Warning: Could not find positions for {len(missing_positions)} players. They will be excluded.")
        print("Missing players:", missing_positions['Name'].tolist())
        
        # Drop players for whom we couldn't find a position
        merged_df.dropna(subset=['Position'], inplace=True)

    return merged_df

In [4]:
def add_player_positions(gameweek_df: pd.DataFrame) -> pd.DataFrame:
    """
    Enriches a gameweek DataFrame with player positions by fetching live data
    from the FPL API and performing a FLEXIBLE, normalized merge on 'Name' ONLY.

    This approach trusts the 'Team' provided in the user's gameweek file and
    solely uses the API to look up the player's position.
    """
    # --- STEP 1: FETCH LIVE DATA AND CREATE A POSITIONS MAP ---
    print("Fetching latest player data from the FPL API...")
    FPL_API_URL = "https://fantasy.premierleague.com/api/bootstrap-static/"
    try:
        response = requests.get(FPL_API_URL)
        response.raise_for_status()
        data = response.json()
        print("API data successfully fetched.")
    except requests.exceptions.RequestException as e:
        print(f"Error: Could not fetch data from the FPL API: {e}")
        return None

    # --- STEP 2: PROCESS API DATA INTO A CLEAN, DE-DUPLICATED DATAFRAME ---
    position_map = {pos['id']: pos['singular_name_short'].replace('GKP', 'GK') for pos in data['element_types']}
    
    player_data_from_api = []
    for player in data['elements']:
        player_data_from_api.append({
            'Name': player['web_name'],
            'Position': position_map.get(player['element_type'], 'Unknown')
        })
    positions_df = pd.DataFrame(player_data_from_api)

    # --- STEP 3: NORMALIZE NAMES AND DE-DUPLICATE THE API DATA ---
    # This is the key step to prevent the row inflation issue from earlier.
    # We create a unique mapping from a player's name to their position.
    positions_df['normalized_name'] = positions_df['Name'].apply(unidecode)
    positions_df.drop_duplicates(subset=['normalized_name'], keep='first', inplace=True)
    
    # --- STEP 4: MERGE ON NAME ONLY ---
    gameweek_df['normalized_name'] = gameweek_df['Name'].apply(unidecode)

    # We select only the columns we need from the positions_df for the merge
    merged_df = pd.merge(
        gameweek_df,
        positions_df[['normalized_name', 'Position']],
        on='normalized_name',
        how='left'
    )

    # --- STEP 5: CLEANUP AND VALIDATE ---
    merged_df.drop(columns=['normalized_name'], inplace=True, errors='ignore')
    
    missing_positions = merged_df[merged_df['Position'].isnull()]
    if not missing_positions.empty:
        print(f"\nWarning: Could not find a position for {len(missing_positions)} players. They will be excluded.")
        print("Missing players:", missing_positions['Name'].values.tolist())
        merged_df.dropna(subset=['Position'], inplace=True)
    
    print(f"Successfully merged positions. Final DataFrame has {len(merged_df)} players.")
    return merged_df


In [5]:
# Define all the files you want to process in a dictionary
gameweek_files = {
    'current_gw_df': 'CurrentGWpredpoints.csv',
    'gw2_df': 'Next2GWpredpoints.csv',
    'gw3_df': 'Next3GWpredpoints.csv',
    'gw4_df': 'Next4GWpredpoints.csv',
    'gw5_df': 'Next5GWpredpoints.csv',
    'gw6_df': 'Next6GWpredpoints.csv'
}

# Create an empty dictionary to store the final, processed DataFrames
fpl_dataframes = {}

print("--- Starting Data Loading Process (Loop Method) ---")

# Loop through each file, process it, and store the result
for df_name, file_path in gameweek_files.items():
    print(f"\nProcessing file: {file_path}...")
    
    # Parse the raw data from the file
    raw_df = parse_fpl_data_from_file(file_path)
    
    # Only proceed if the file was parsed successfully
    if raw_df is not None:
        # Add positions and store the final DataFrame in our dictionary
        fpl_dataframes[df_name] = add_player_positions(raw_df)

print("\n--- All Gameweek DataFrames Loaded Successfully ---")

# For example, to check the head of the 6-week data:
if 'gw6_df' in fpl_dataframes and fpl_dataframes['gw6_df'] is not None:
    print("\nSample of GW6 DataFrame:")
    print(fpl_dataframes['gw6_df'].head())

--- Starting Data Loading Process (Loop Method) ---

Processing file: CurrentGWpredpoints.csv...
Successfully parsed 400 players.
Fetching latest player data from the FPL API...
API data successfully fetched.
Successfully merged positions. Final DataFrame has 400 players.

Processing file: Next2GWpredpoints.csv...
Successfully parsed 400 players.
Fetching latest player data from the FPL API...
API data successfully fetched.
Successfully merged positions. Final DataFrame has 400 players.

Processing file: Next3GWpredpoints.csv...
Successfully parsed 400 players.
Fetching latest player data from the FPL API...
API data successfully fetched.
Successfully merged positions. Final DataFrame has 400 players.

Processing file: Next4GWpredpoints.csv...
Successfully parsed 400 players.
Fetching latest player data from the FPL API...
API data successfully fetched.
Successfully merged positions. Final DataFrame has 400 players.

Processing file: Next5GWpredpoints.csv...
Successfully parsed 400 pla

## Optimizing team for current gameweek


#### Mathematical Formulation of the FPL Team Optimization Problem

This document outlines the Mixed-Integer Linear Program (MILP) used to select an optimal Fantasy Premier League (FPL) starting eleven, including a captain.

---

#### 1. Sets and Indices

Let $P$ be the set of all available players in the game. We define the following subsets based on player position:
- $G \subset P$: Set of Goalkeepers
- $D \subset P$: Set of Defenders
- $M \subset P$: Set of Midfielders
- $F \subset P$: Set of Forwards

Furthermore, let $T_k \subset P$ be the set of players belonging to a specific Premier League team $k$.

---

#### 2. Parameters

These are the known inputs to our model for each player $i \in P$:
- $Points_i$: The predicted points for player $i$.
- $Price_i$: The cost of player $i$.
- $B_{total}$: The total available budget for the 11 starting players.

---

#### 3. Decision Variables

We define two sets of binary decision variables for each player $i \in P$:

- $x_i \in \{0, 1\}$: This variable determines if a player is selected for the team.
  - $x_i = 1$ if player $i$ is in the starting eleven.
  - $x_i = 0$ otherwise.

- $c_i \in \{0, 1\}$: This variable determines if a player is chosen as the team captain.
  - $c_i = 1$ if player $i$ is the captain.
  - $c_i = 0$ otherwise.

---

#### 4. Objective Function

The objective is to **maximize** the total predicted points of the starting eleven, accounting for the captain's doubled score. The total score $Z$ is the sum of the base points of all selected players plus the bonus points for the captain.

$$
\text{Maximize} \quad Z = \sum_{i \in P} (Points_i \cdot x_i) + \sum_{i \in P} (Points_i \cdot c_i)
$$

This can also be expressed as:

$$
\text{Maximize} \quad Z = \sum_{i \in P} Points_i \cdot (x_i + c_i)
$$

---

#### 5. Constraints

The solution must adhere to the following FPL game rules:

**a) Squad Size Constraint:** The team must consist of exactly 11 players.
$$
\sum_{i \in P} x_i = 11
$$

**b) Budget Constraint:** The total cost of the selected players must not exceed the available budget.
$$
\sum_{i \in P} Price_i \cdot x_i \le B_{total}
$$

**c) Captaincy Constraints:**
- Exactly one player must be chosen as captain.
$$
\sum_{i \in P} c_i = 1
$$
- A player can only be captain if they are selected in the starting eleven. This links the two decision variables.
$$
c_i \le x_i \quad \forall i \in P
$$

**d) Positional Constraints:** The squad must meet the formation requirements.
- **Goalkeepers:** Exactly 1 goalkeeper must be selected.
$$
\sum_{i \in G} x_i = 1
$$
- **Defenders:** Between 3 and 5 defenders must be selected.
$$
\sum_{i \in D} x_i \ge 3
$$
$$
\sum_{i \in D} x_i \le 5
$$
- **Midfielders:** Between 2 and 5 midfielders must be selected.
$$
\sum_{i \in M} x_i \ge 2
$$
$$
\sum_{i \in M} x_i \le 5
$$
- **Forwards:** Between 1 and 3 forwards must be selected.
$$
\sum_{i \in F} x_i \ge 1
$$
$$
\sum_{i \in F} x_i \le 3
$$

**e) Team Constraint:** A maximum of 3 players can be selected from any single Premier League team. For each team $k$:
$$
\sum_{i \in T_k} x_i \le 3
$$

In [6]:
def optimize_fpl_team(player_df: pd.DataFrame, team_value: float, bench_cost: float):
    """
    Optimizes an FPL starting 11, including a captain, based on predicted points.

    Args:
        player_df: DataFrame with player data. Must include columns:
                   'Name', 'Team', 'Predicted Points', 'Price', 'Position'.
        team_value: The total value of your team (e.g., 100.0).
        bench_cost: The combined cost of your 4 bench players.
                    (e.g., 4.0 + 4.5 + 4.0 + 4.0 = 16.5)

    Returns:
        A pandas DataFrame containing the 11 players of the optimal team,
        or None if no solution is found.
    """
    if 'Position' not in player_df.columns:
        raise ValueError("Input DataFrame must contain a 'Position' column.")

    available_budget = team_value - bench_cost
    players = player_df.to_dict('records')
    player_indices = range(len(players))

    # 1. INITIALIZE THE MILP PROBLEM
    model = LpProblem("FPL_Team_Optimization_with_Captain", LpMaximize)

    # 2. DEFINE DECISION VARIABLES
    # 'player_vars': Binary variable, 1 if player is in the team, 0 otherwise
    player_vars = LpVariable.dicts("Player", player_indices, cat='Binary')
    
    # --- NEW: Captain Variables ---
    # 'captain_vars': Binary variable, 1 if player is the captain, 0 otherwise
    captain_vars = LpVariable.dicts("Captain", player_indices, cat='Binary')

    # 3. DEFINE THE OBJECTIVE FUNCTION
    # --- MODIFIED: Include Captain Points ---
    # The total score is the sum of points of all players in the team,
    # PLUS an additional sum of points for the player chosen as captain.
    # This correctly doubles the captain's score.
    model += lpSum(
        players[i]['Predicted Points'] * player_vars[i] + 
        players[i]['Predicted Points'] * captain_vars[i] 
        for i in player_indices
    ), "Total Predicted Points"

    # 4. DEFINE THE CONSTRAINTS
    
    # -- Budget & Squad Size Constraints (Unchanged) --
    model += lpSum(players[i]['Price'] * player_vars[i] for i in player_indices) <= available_budget, "Budget Constraint"
    model += lpSum(player_vars[i] for i in player_indices) == 11, "Squad Size Constraint"
    
    # --- NEW: Captaincy Constraints ---
    # a) We must select exactly one captain from the squad.
    model += lpSum(captain_vars[i] for i in player_indices) == 1, "Captain Count Constraint"

    # b) A player can only be captain IF they are in the starting 11.
    #    This links the two sets of variables together.
    for i in player_indices:
        model += captain_vars[i] <= player_vars[i], f"Captain in Team Constraint_{i}"

    # -- Positional Constraints (Unchanged) --
    gks = [i for i in player_indices if players[i]['Position'] == 'GK']
    defs = [i for i in player_indices if players[i]['Position'] == 'DEF']
    mids = [i for i in player_indices if players[i]['Position'] == 'MID']
    fwds = [i for i in player_indices if players[i]['Position'] == 'FWD']

    model += lpSum(player_vars[i] for i in gks) == 1, "Goalkeeper Constraint"
    model += lpSum(player_vars[i] for i in defs) >= 3, "Min Defenders"
    model += lpSum(player_vars[i] for i in defs) <= 5, "Max Defenders"
    model += lpSum(player_vars[i] for i in mids) >= 2, "Min Midfielders"
    model += lpSum(player_vars[i] for i in mids) <= 5, "Max Midfielders"
    model += lpSum(player_vars[i] for i in fwds) >= 1, "Min Forwards"
    model += lpSum(player_vars[i] for i in fwds) <= 3, "Max Forwards"

    # -- Team Constraint (Unchanged) --
    team_names = player_df['Team'].unique()
    for team in team_names:
        team_players = [i for i in player_indices if players[i]['Team'] == team]
        model += lpSum(player_vars[i] for i in team_players) <= 3, f"Team Constraint {team}"
        
    # 5. SOLVE THE PROBLEM
    model.solve()

    # 6. DISPLAY THE RESULTS (MODIFIED)
    if LpStatus[model.status] == 'Optimal':
        print(f"Optimal solution found!")
        
        selected_indices = [i for i in player_indices if player_vars[i].value() == 1]
        # --- NEW: Identify the chosen captain ---
        captain_index = [i for i in player_indices if captain_vars[i].value() == 1][0]
        
        optimal_team_df = player_df.iloc[selected_indices].copy()
        
        # Add a column to clearly mark the captain
        optimal_team_df['Role'] = ''
        optimal_team_df.loc[optimal_team_df.index == captain_index, 'Role'] = '(C)'

        # --- MODIFIED: Calculate total points correctly ---
        base_points = optimal_team_df['Predicted Points'].sum()
        captain_points = optimal_team_df.loc[optimal_team_df.index == captain_index, 'Predicted Points'].iloc[0]
        total_points_with_captain = base_points + captain_points
        
        total_cost = optimal_team_df['Price'].sum()

        print(f"\nTotal Predicted Points (with Captain): {total_points_with_captain:.2f}")
        print(f"Total Cost: £{total_cost:.1f}m\n")

        # Reorder columns for better display
        optimal_team_df = optimal_team_df[['Name', 'Role', 'Team', 'Position', 'Price', 'Predicted Points']]
        
        # Print the team sorted by position
        for position in ['GK', 'DEF', 'MID', 'FWD']:
            print(f"--- {position}S ---")
            print(optimal_team_df[optimal_team_df['Position'] == position].to_string(index=False))
            print()
            
        return optimal_team_df
    else:
        print(f"Could not find an optimal solution. Status: {LpStatus[model.status]}")
        return None

##### Optimize for current game week: Free hit draft

In [7]:
MY_TEAM_VALUE = 100  # Your total budget including money in the bank
    
MINIMAL_BENCH_COST = 4.0 + 4.0 + 4.0 + 4.5 
    
# Run the optimization for the i-gameweek horizon
print("--- Optimizing Team for the current Gameweek ---")
optimal_1_gw_team = optimize_fpl_team(
        player_df=fpl_dataframes['current_gw_df'],
        team_value=MY_TEAM_VALUE,
        bench_cost=MINIMAL_BENCH_COST
    )

--- Optimizing Team for the current Gameweek ---
Optimal solution found!

Total Predicted Points (with Captain): 59.50
Total Cost: £83.4m

--- GKS ---
Name Role   Team Position  Price  Predicted Points
Leno      Fulham       GK    5.0               3.7

--- DEFS ---
      Name Role           Team Position  Price  Predicted Points
    Virgil           Liverpool      DEF    6.1               4.6
     Munoz      Crystal Palace      DEF    5.6               4.5
  Andersen              Fulham      DEF    4.5               4.2
N.Williams        Notts Forest      DEF    5.0               4.2

--- MIDS ---
   Name Role        Team Position  Price  Predicted Points
M.Salah  (C)   Liverpool      MID   14.5               6.5
  Gakpo        Liverpool      MID    7.7               5.1
Semenyo      Bournemouth      MID    7.5               4.9
 Mbeumo          Man Utd      MID    8.1               4.7
 Palmer          Chelsea      MID   10.4               5.7

--- FWDS ---
    Name Role    Team Posi

#### Optimize for long-term performance: Next 6 Gameweeks

In [8]:
MY_TEAM_VALUE = 100  # Your total budget including money in the bank
    
MINIMAL_BENCH_COST = 4.0 + 4.0 + 4.0 + 4.5 
    
# Run the optimization for the i-gameweek horizon
print("--- Optimizing Team for the Next 6 Gameweeks (Long-Term View) ---")
optimal_6_gw_team = optimize_fpl_team(
        player_df=fpl_dataframes['gw6_df'],
        team_value=MY_TEAM_VALUE,
        bench_cost=MINIMAL_BENCH_COST
    )

--- Optimizing Team for the Next 6 Gameweeks (Long-Term View) ---
Optimal solution found!

Total Predicted Points (with Captain): 342.60
Total Cost: £83.0m

--- GKS ---
Name Role    Team Position  Price  Predicted Points
Raya      Arsenal       GK    5.5              23.7

--- DEFS ---
     Name Role           Team Position  Price  Predicted Points
   Virgil           Liverpool      DEF    6.1              26.2
    Munoz      Crystal Palace      DEF    5.6              25.6
Tarkowski             Everton      DEF    5.5              24.8
    Keane             Everton      DEF    4.5              23.4

--- MIDS ---
       Name Role        Team Position  Price  Predicted Points
    M.Salah  (C)   Liverpool      MID   14.5              37.4
B.Fernandes          Man Utd      MID    9.0              29.6
     Mbeumo          Man Utd      MID    8.1              29.3
      Gakpo        Liverpool      MID    7.7              28.8
    Semenyo      Bournemouth      MID    7.5              28.7



## Optimizing for Use of Transfers

#### Make Gameweek Predicted Points Dataframe

In [9]:
def create_per_gameweek_df(cumulative_dfs: dict) -> pd.DataFrame:
    """
    Transforms a dictionary of cumulative-point DataFrames into a single,
    master DataFrame with predicted points for each individual gameweek.

    (REVISED to a STRICTER method: This version now only includes players who are
    present in ALL provided gameweek projection files, ensuring data integrity.)

    Args:
        cumulative_dfs: A dictionary where keys are DataFrame names (e.g., 'current_gw_df')
                        and values are the corresponding DataFrames.
    Returns:
        A pandas DataFrame containing only players with complete projections.
    """
    if not cumulative_dfs or 'gw6_df' not in cumulative_dfs:
        print("Error: Input dictionary is empty or missing the 'gw6_df' key.")
        return None

    # --- Step 1: Initialize the master DataFrame and de-duplicate it ---
    # Start with the longest-horizon DataFrame as the base.
    master_df = cumulative_dfs['gw6_df'][['Name', 'Team', 'Position', 'Price', 'Predicted Points']].copy()
    master_df.rename(columns={'Predicted Points': 'GW6_Cum_Points'}, inplace=True)
    master_df.drop_duplicates(subset=['Name'], keep='first', inplace=True)
    
    initial_player_count = len(master_df)
    print(f"Starting with a base of {initial_player_count} unique players from the GW6 file.")

    # --- Step 2: Iteratively perform INNER merges to find common players ---
    # We process the other DataFrames in reverse order (GW5 down to GW1).
    df_keys_in_order = ['gw5_df', 'gw4_df', 'gw3_df', 'gw2_df', 'current_gw_df']
    
    for i, key in enumerate(df_keys_in_order):
        gw_num = 5 - i # Corresponding gameweek number
        
        if key not in cumulative_dfs or cumulative_dfs[key] is None:
            print(f"Error: DataFrame for '{key}' not found. Cannot proceed with strict filtering.")
            return None # In strict mode, missing files are a fatal error
            
        points_df = cumulative_dfs[key][['Name', 'Predicted Points']]
        points_df.drop_duplicates(subset=['Name'], keep='first', inplace=True)
        points_df = points_df.rename(columns={'Predicted Points': f'GW{gw_num}_Cum_Points'})
        
        # --- THE KEY CHANGE ---
        # An 'inner' merge only keeps players that exist in BOTH DataFrames.
        # By chaining these, we filter down to the set of players present in all files.
        master_df = pd.merge(master_df, points_df, on='Name', how='inner')

    print(f"Filtered down to {len(master_df)} players present in all gameweek files.")

    # --- Step 3: Calculate the individual gameweek points ---
    # The columns are now guaranteed to be present and have no missing values.
    master_df['GW1_Points'] = master_df['GW1_Cum_Points']
    for i in range(2, 7):
        master_df[f'GW{i}_Points'] = master_df[f'GW{i}_Cum_Points'] - master_df[f'GW{i-1}_Cum_Points']
        master_df[f'GW{i}_Points'] = master_df[f'GW{i}_Points'].clip(lower=0)

    # --- Step 4: Final cleanup ---
    static_cols = ['Name', 'Team', 'Position', 'Price']
    points_cols = [f'GW{i}_Points' for i in range(1, 7)]
    final_df = master_df[static_cols + points_cols]

    print("Master per-gameweek DataFrame created successfully with high-integrity data.")
    return final_df

In [10]:
# 1. Load all gameweek files into a dictionary
gameweek_files = {
        'current_gw_df': 'CurrentGWpredpoints.csv',
        'gw2_df': 'Next2GWpredpoints.csv',
        'gw3_df': 'Next3GWpredpoints.csv',
        'gw4_df': 'Next4GWpredpoints.csv',
        'gw5_df': 'Next5GWpredpoints.csv',
        'gw6_df': 'Next6GWpredpoints.csv'
    }
    
fpl_dataframes = {}
print("--- Loading and Processing All Gameweek Files ---")
for df_name, file_path in gameweek_files.items():
    print(f"Processing {file_path}...")
    raw_df = parse_fpl_data_from_file(file_path)
    if raw_df is not None:
        fpl_dataframes[df_name] = add_player_positions(raw_df)

# 2. Create the final, per-gameweek master DataFrame
print("\n--- Creating the Master Per-Gameweek DataFrame ---")
master_fpl_df = create_per_gameweek_df(fpl_dataframes)

# 3. Inspect the final result
if master_fpl_df is not None:
    print("\n--- Final DataFrame Info ---")
    master_fpl_df.info()
        
    print("\n--- Final DataFrame Head ---")
    print(master_fpl_df.head())

--- Loading and Processing All Gameweek Files ---
Processing CurrentGWpredpoints.csv...
Successfully parsed 400 players.
Fetching latest player data from the FPL API...
API data successfully fetched.
Successfully merged positions. Final DataFrame has 400 players.
Processing Next2GWpredpoints.csv...
Successfully parsed 400 players.
Fetching latest player data from the FPL API...
API data successfully fetched.
Successfully merged positions. Final DataFrame has 400 players.
Processing Next3GWpredpoints.csv...
Successfully parsed 400 players.
Fetching latest player data from the FPL API...
API data successfully fetched.
Successfully merged positions. Final DataFrame has 400 players.
Processing Next4GWpredpoints.csv...
Successfully parsed 400 players.
Fetching latest player data from the FPL API...
API data successfully fetched.
Successfully merged positions. Final DataFrame has 400 players.
Processing Next5GWpredpoints.csv...
Successfully parsed 400 players.
Fetching latest player data fro

C:\Users\cleme\AppData\Local\Temp\ipykernel_14876\3098413297.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points_df.drop_duplicates(subset=['Name'], keep='first', inplace=True)
C:\Users\cleme\AppData\Local\Temp\ipykernel_14876\3098413297.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points_df.drop_duplicates(subset=['Name'], keep='first', inplace=True)
C:\Users\cleme\AppData\Local\Temp\ipykernel_14876\3098413297.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [11]:
# Use this one-liner to find an exact player name in final DataFrame
master_fpl_df[master_fpl_df['Name'] == 'King']

,Name,Team,Position,Price,GW1_Points,GW2_Points,GW3_Points,GW4_Points,GW5_Points,GW6_Points
149,King,Fulham,GK,4.5,3.2,2.8,2.7,2.5,2.5,3.2


### Multi-Period Optimization 

### Mathematical Formulation of the Multi-Week FPL Transfer Optimizer

This document outlines the Mixed-Integer Linear Program (MILP) for determining an optimal FPL transfer strategy over a multi-week horizon.

---

#### 1. Sets and Indices

- $P$: The set of all available players, indexed by $i$.
- $W$: The set of gameweeks in the optimization horizon (e.g., $\{0, 1, ..., 5\}$ for a 6-week horizon), indexed by $w$.
- $G, D, M, F \subset P$: Subsets of players for Goalkeepers, Defenders, Midfielders, and Forwards.
- $T_k \subset P$: Subset of players belonging to a specific Premier League team $k$.

---

#### 2. Parameters

**Player-Specific Parameters:**
- $Points_{i,w}$: The predicted points for player $i$ in gameweek $w$.
- $Price_i$: The market price of player $i$.

**Manager-Specific Parameters (Inputs):**
- $Capital_{total}$: The total capital (spending power) of the FPL manager.
- $Squad_{i, initial} \in \{0, 1\}$: An indicator of whether player $i$ is in the manager's initial squad.
- $FT_{initial}$: The number of free transfers available at the start of the horizon.

**Strategic Parameters:**
- $V_k$: The pre-calculated cumulative point value of finishing the horizon with exactly $k$ banked free transfers.

---

#### 3. Decision Variables

**Weekly Squad and Team Selection Variables (for each week $w \in W$ and player $i \in P$):**
- $InTeam_{i,w} \in \{0, 1\}$: 1 if player $i$ is in the 15-man squad in week $w$, 0 otherwise.
- $Start_{i,w} \in \{0, 1\}$: 1 if player $i$ is in the starting 11 in week $w$, 0 otherwise.
- $Bench_{i,w} \in \{0, 1\}$: 1 if player $i$ is on the bench in week $w$, 0 otherwise.
- $Captain_{i,w} \in \{0, 1\}$: 1 if player $i$ is the captain in week $w$, 0 otherwise.

**Weekly Transfer Variables (for each week $w \in W$ and player $i \in P$):**
- $Buy_{i,w} \in \{0, 1\}$: 1 if player $i$ is bought in week $w$, 0 otherwise.
- $Sell_{i,w} \in \{0, 1\}$: 1 if player $i$ is sold in week $w$, 0 otherwise.

**Weekly Transfer Management Variables (for each week $w \in W$):**
- $Transfers_{w} \in \mathbb{Z}^+$: Total number of transfers made in week $w$.
- $PaidTransfers_{w} \in \mathbb{Z}^+$: Number of transfers that incur a point hit in week $w$.
- $FreeUsed_{w} \in \mathbb{Z}^+$: Number of free transfers used in week $w$.
- $FT_{w} \in \{1, ..., 5\}$: Number of free transfers available at the start of week $w$.

**Terminal State Variables (for the end of the horizon):**
- $FT_{future} \in \{1, ..., 5\}$: The number of free transfers available *after* the final week's transfers.
- $FutureState_k \in \{0, 1\}$: 1 if $FT_{future} = k$, 0 otherwise.

---

#### 4. Objective Function

The objective is to **maximize** the total expected value, which is the sum of all points scored over the horizon, minus transfer costs, plus a terminal value for banked free transfers.

$$
\text{Maximize} \quad Z = \left( \sum_{w \in W} \sum_{i \in P} Points_{i,w} \cdot (Start_{i,w} + Captain_{i,w}) \right) - \left( \sum_{w \in W} 4 \cdot PaidTransfers_{w} \right) + \left( \sum_{k=1}^{5} V_k \cdot FutureState_k \right)
$$

---

#### 5. Constraints

**a) Squad and Team Structure (for each week $w \in W$):**
- **Total Capital:** $\sum_{i \in P} Price_i \cdot InTeam_{i,w} \le Capital_{total}$
- **Squad Size:** $\sum_{i \in P} InTeam_{i,w} = 15$
- **Squad Positions:** $\sum_{i \in G} InTeam_{i,w} = 2$, $\sum_{i \in D} InTeam_{i,w} = 5$, $\sum_{i \in M} InTeam_{i,w} = 5$, $\sum_{i \in F} InTeam_{i,w} = 3$
- **Starting XI Size:** $\sum_{i \in P} Start_{i,w} = 11$
- **Captain:** $\sum_{i \in P} Captain_{i,w} = 1$
- **Team Limit:** $\sum_{i \in T_k} InTeam_{i,w} \le 3$ for each team $k$.
- **Starting XI Formation:** Constraints on the number of starters from each position (e.g., $\sum_{i \in G} Start_{i,w} = 1$, etc.)

**b) Linking Constraints (for each week $w \in W$ and player $i \in P$):**
- **Starter/Bench:** $Start_{i,w} + Bench_{i,w} = InTeam_{i,w}$
- **Captain in XI:** $Captain_{i,w} \le Start_{i,w}$

**c) Transfer Logic (for each week $w \in W$):**
- **Counting Transfers:** $\sum_{i \in P} Buy_{i,w} = Transfers_w$ and $\sum_{i \in P} Sell_{i,w} = Transfers_w$
- **Like-for-Like Transfers:** $\sum_{i \in G} Buy_{i,w} = \sum_{i \in G} Sell_{i,w}$ (and similarly for D, M, F).
- **Transfer Cost:** $Transfers_w = FreeUsed_w + PaidTransfers_w$
- **Free Transfer Limit:** $FreeUsed_w \le FT_w$

**d) Week-to-Week Dynamics (for weeks $w \in \{1, ..., |W|-1\}$):**
- **Squad Continuity:** $InTeam_{i,w} = InTeam_{i, w-1} - Sell_{i,w} + Buy_{i,w}$ for each player $i$.
- **Free Transfer Generation:** $FT_w \le (FT_{w-1} - FreeUsed_{w-1}) + 1$ and $FT_w \le 5$.

**e) Initial State (for week $w=0$):**
- **Evolution Mode:** $InTeam_{i,0} = Squad_{i, initial} - Sell_{i,0} + Buy_{i,0}$
- **Initialization Mode:** $Buy_{i,0} = 0$, $Sell_{i,0} = 0$. The solver directly chooses $InTeam_{i,0}$.
- **Initial FTs:** $FT_0 = FT_{initial}$

**f) Terminal State (after the final week $w_{end}$):**
- **Future FT Calculation:** $FT_{future} \le (FT_{w_{end}} - FreeUsed_{w_{end}}) + 1$ and $FT_{future} \le 5$.
- **State Selection:** $\sum_{k=1}^{5} FutureState_k = 1$
- **Linking Future FTs to State:** $FT_{future} = \sum_{k=1}^{5} k \cdot FutureState_k$

In [12]:
def optimize_transfers_for_horizon(
    player_df: pd.DataFrame,
    current_squad: list = None,
    bank: float = 100.0,
    free_transfers_available: int = 1,
    horizon: int = 6,
    ft_base_value: float = 4.0,
    ft_decay: float = 0.3
):
    MAX_FREE_TRANSFERS = 5

    # --- 1. SETUP AND DATA PREPARATION ---
    weeks = range(horizon)
    players = player_df.to_dict('records')
    player_indices = range(len(players))
    name_to_index = {player['Name']: i for i, player in enumerate(players)}

    model = LpProblem(f"FPL_Transfer_Strategy_Horizon_{horizon}", LpMaximize)

    # --- 2. DEFINE DECISION VARIABLES ---
    in_team = LpVariable.dicts("in_team", (weeks, player_indices), cat='Binary')
    starting = LpVariable.dicts("starting", (weeks, player_indices), cat='Binary')
    benched = LpVariable.dicts("benched", (weeks, player_indices), cat='Binary')
    captain = LpVariable.dicts("captain", (weeks, player_indices), cat='Binary')
    buy = LpVariable.dicts("buy", (weeks, player_indices), cat='Binary')
    sell = LpVariable.dicts("sell", (weeks, player_indices), cat='Binary')
    transfers_made = LpVariable.dicts("transfers_made", weeks, lowBound=0, cat='Integer')
    paid_transfers = LpVariable.dicts("paid_transfers", weeks, lowBound=0, cat='Integer')
    free_transfers_used = LpVariable.dicts("free_transfers_used", weeks, lowBound=0, cat='Integer')
    ft_available = LpVariable.dicts("ft_available", weeks, lowBound=1, upBound=MAX_FREE_TRANSFERS, cat='Integer')
    
    # --- NEW: Variable to represent FTs available AFTER the horizon ---
    ft_for_future = LpVariable("ft_for_future", lowBound=1, upBound=MAX_FREE_TRANSFERS, cat='Integer')
    future_ft_state = LpVariable.dicts("future_ft_state", range(1, MAX_FREE_TRANSFERS + 1), cat='Binary')

    # --- 3. DEFINE THE OBJECTIVE FUNCTION ---
    total_points = lpSum(
        players[i][f'GW{w+1}_Points'] * starting[w][i] +
        players[i][f'GW{w+1}_Points'] * captain[w][i]
        for w in weeks for i in player_indices
    )
    total_hits_cost = lpSum(4 * paid_transfers[w] for w in weeks)
    
    # Terminal value is based on the future FT state ---
    ft_values = {k: sum(max(0, ft_base_value - (i - 1) * ft_decay) for i in range(1, k + 1)) for k in range(1, MAX_FREE_TRANSFERS + 1)}
    end_of_horizon_ft_value = lpSum(ft_values[k] * future_ft_state[k] for k in range(1, MAX_FREE_TRANSFERS + 1))
    
    model += total_points - total_hits_cost + end_of_horizon_ft_value, "Total_Expected_Value"

    # --- 4. DEFINE CONSTRAINTS ---
    if current_squad is None:
        total_capital = bank
        for i in player_indices:
            model += buy[0][i] == 0; model += sell[0][i] == 0
    else:
        current_squad_indices = [name_to_index.get(name) for name in current_squad if name in name_to_index]
        initial_squad_flags = {i: 1 if i in current_squad_indices else 0 for i in player_indices}
        for i in player_indices:
            model += in_team[0][i] == initial_squad_flags[i] - sell[0][i] + buy[0][i]
        initial_squad_value = sum(players[i]['Price'] for i in current_squad_indices)
        total_capital = initial_squad_value + bank
    
    model += ft_available[0] == free_transfers_available

    gks = [i for i in player_indices if players[i]['Position'] == 'GK']
    defs = [i for i in player_indices if players[i]['Position'] == 'DEF']
    mids = [i for i in player_indices if players[i]['Position'] == 'MID']
    fwds = [i for i in player_indices if players[i]['Position'] == 'FWD']

    for w in weeks:
        model += lpSum(players[i]['Price'] * in_team[w][i] for i in player_indices) <= total_capital
        model += lpSum(in_team[w][i] for i in player_indices) == 15
        model += lpSum(in_team[w][i] for i in gks) == 2
        model += lpSum(in_team[w][i] for i in defs) == 5
        model += lpSum(in_team[w][i] for i in mids) == 5
        model += lpSum(in_team[w][i] for i in fwds) == 3
        model += lpSum(starting[w][i] for i in player_indices) == 11
        model += lpSum(captain[w][i] for i in player_indices) == 1
        model += lpSum(starting[w][i] for i in gks) == 1
        model += lpSum(starting[w][i] for i in defs) >= 3
        model += lpSum(starting[w][i] for i in defs) <= 5
        model += lpSum(starting[w][i] for i in mids) >= 2
        model += lpSum(starting[w][i] for i in mids) <= 5
        model += lpSum(starting[w][i] for i in fwds) >= 1
        model += lpSum(starting[w][i] for i in fwds) <= 3
        for team in player_df['Team'].unique():
            team_players = [i for i in player_indices if players[i]['Team'] == team]
            model += lpSum(in_team[w][i] for i in team_players) <= 3
        for i in player_indices:
            model += starting[w][i] + benched[w][i] == in_team[w][i]
            model += captain[w][i] <= starting[w][i]
        if w > 0 or current_squad is not None:
            model += lpSum(buy[w][i] for i in gks) == lpSum(sell[w][i] for i in gks)
            model += lpSum(buy[w][i] for i in defs) == lpSum(sell[w][i] for i in defs)
            model += lpSum(buy[w][i] for i in mids) == lpSum(sell[w][i] for i in mids)
            model += lpSum(buy[w][i] for i in fwds) == lpSum(sell[w][i] for i in fwds)
        model += lpSum(buy[w][i] for i in player_indices) == transfers_made[w]
        model += lpSum(sell[w][i] for i in player_indices) == transfers_made[w]
        model += transfers_made[w] == free_transfers_used[w] + paid_transfers[w]
        model += free_transfers_used[w] <= ft_available[w]

    for w in range(1, horizon):
        uncapped_ft = ft_available[w-1] - free_transfers_used[w-1] + 1
        model += ft_available[w] <= uncapped_ft
        model += ft_available[w] <= MAX_FREE_TRANSFERS
        for i in player_indices:
            model += in_team[w][i] == in_team[w-1][i] - sell[w][i] + buy[w][i]

    # Constraint to calculate the number of FTs for the future ---
    # This correctly calculates the transfers available for GW7 (the week after our horizon).
    final_week_index = horizon - 1
    uncapped_future_ft = ft_available[final_week_index] - free_transfers_used[final_week_index] + 1
    model += ft_for_future <= uncapped_future_ft
    model += ft_for_future <= MAX_FREE_TRANSFERS

    # Link this new variable to the binary state variables for the objective function
    model += lpSum(future_ft_state[k] for k in range(1, MAX_FREE_TRANSFERS + 1)) == 1
    model += ft_for_future == lpSum(k * future_ft_state[k] for k in range(1, MAX_FREE_TRANSFERS + 1))
    
    # --- 5. SOLVE THE PROBLEM ---
    print("Solver is running... This might take a few moments.")
    model.solve()

    # --- 6. PROCESS AND DISPLAY RESULTS ---
    if LpStatus[model.status] == 'Optimal':
        total_score = model.objective.value()
        print(f"\nOptimal strategy found! Total Expected Value over {horizon} weeks: {total_score:.2f}")

        # (Results display logic is the same)
        initial_squad_indices = [i for i in player_indices if in_team[0][i].value() > 0.5]
        initial_squad_df = player_df.iloc[initial_squad_indices].copy()
        roles = []
        for i in initial_squad_df.index:
            if captain[0][i].value() > 0.5:
                roles.append('(C)')
            elif starting[0][i].value() > 0.5:
                roles.append('')
            else:
                roles.append('Bench')
        initial_squad_df['Role'] = roles
        initial_squad_value = initial_squad_df['Price'].sum()
        money_remaining = total_capital - initial_squad_value
        print("\n" + "="*50)
        print(f"--- SQUAD FOR GAMEWEEK 1 ---")
        print(f"Total Squad Value: £{initial_squad_value:.1f}m | Money Remaining: £{money_remaining:.1f}m")
        print("="*50)
        starters_df = initial_squad_df[initial_squad_df['Role'] != 'Bench'].copy()
        bench_df = initial_squad_df[initial_squad_df['Role'] == 'Bench'].copy()
        pos_order = ['GK', 'DEF', 'MID', 'FWD']
        starters_df['Position'] = pd.Categorical(starters_df['Position'], categories=pos_order, ordered=True)
        starters_df = starters_df.sort_values('Position')
        print("--- Starting XI ---")
        print(starters_df[['Name', 'Role', 'Team', 'Position', 'Price']].to_string(index=False))
        bench_df['Position'] = pd.Categorical(bench_df['Position'], categories=pos_order, ordered=True)
        bench_df = bench_df.sort_values('Position')
        print("\n--- Bench ---")
        print(bench_df[['Name', 'Team', 'Position', 'Price']].to_string(index=False))
        print("="*50)
        print("\n--- RECOMMENDED ACTIONS FOR HORIZON ---")
        actions = []
        for w in weeks:
            transfers_in = [players[i]['Name'] for i in player_indices if buy[w][i].value() > 0.5]
            transfers_out = [players[i]['Name'] for i in player_indices if sell[w][i].value() > 0.5]
            week_actions = f"--- Gameweek {w+1} ---"
            if w == 0 and current_squad is None:
                week_actions += "\nAction: Build initial squad."
            elif transfers_in:
                week_actions += f"\nTransfers: {len(transfers_in)}."
                sold_map = {tout: player_df[player_df['Name'] == tout].iloc[0] for tout in transfers_out}
                bought_map = {tin: player_df[player_df['Name'] == tin].iloc[0] for tin in transfers_in}
                for pos in ['GK', 'DEF', 'MID', 'FWD']:
                    sold_pos = [name for name, data in sold_map.items() if data['Position'] == pos]
                    bought_pos = [name for name, data in bought_map.items() if data['Position'] == pos]
                    for tin, tout in zip(bought_pos, sold_pos):
                        week_actions += f" Buy {tin} ({pos}), Sell {tout} ({pos})."
                if paid_transfers[w].value() > 0.5:
                    week_actions += f" ({int(paid_transfers[w].value())} hits)"
            else:
                week_actions += "\nAction: Roll transfer."
            captain_player = [players[i]['Name'] for i in player_indices if captain[w][i].value() > 0.5][0]
            week_actions += f"\nCaptain: {captain_player}"
            actions.append(week_actions)
        for action in actions:
            print(action)
            
        return total_score, initial_squad_df, actions
    else:
        print(f"\nCould not find an optimal solution. Status: {LpStatus[model.status]}")
        return None, None, None

### Wildcard Squad and plan 

In [13]:
# --- RUN IN INITIALIZATION MODE (GW1 / WILDCARD) ---
print("--- Running in INITIALIZATION Mode (for GW1) ---")
score, initialScore, plan = optimize_transfers_for_horizon(
        player_df=master_fpl_df,
        current_squad=None,
        bank=100.0
)

--- Running in INITIALIZATION Mode (for GW1) ---
Solver is running... This might take a few moments.

Optimal strategy found! Total Expected Value over 6 weeks: 362.20

--- SQUAD FOR GAMEWEEK 1 ---
Total Squad Value: £99.6m | Money Remaining: £0.4m
--- Starting XI ---
       Name Role           Team Position  Price
       Raya             Arsenal       GK    5.5
     Virgil           Liverpool      DEF    6.1
      Munoz      Crystal Palace      DEF    5.6
  Tarkowski             Everton      DEF    5.5
      Ekdal             Burnley      DEF    4.0
    M.Salah  (C)      Liverpool      MID   14.5
B.Fernandes             Man Utd      MID    9.0
     Mbeumo             Man Utd      MID    8.1
      Gakpo           Liverpool      MID    7.7
    Semenyo         Bournemouth      MID    7.5
    Watkins         Aston Villa      FWD    8.8

--- Bench ---
     Name         Team Position  Price
     John Notts Forest       GK    4.0
    Keane      Everton      DEF    4.5
      Obi      Man Utd 

### Plan for existing squad

In [14]:
# Example 15-man squad list (replace these names with your actual players)
my_team_squad = [
    'Raya', 'Dubravka',
   'Munoz', 'Pedro Porro', 'Cucurella', 'Senesi', 'Livramento',
   'L.Paqueta', 'Eze', 'Semenyo', 'Reijnders', 'Grealish',
  'Haaland', 'Gyokeres', 'Joao Pedro'
]

#AI_team_squad = [
#    'Trafford', 'Dubravka',
#   'Munoz', 'Gabriel', 'Van de Ven', 'Esteve', 'Truffert',
#    'Cunha', 'Kudus', 'Semenyo', 'Rogers', 'Devenny',
#   'Haaland', 'Watkins', 'Gyokeres'
#]

# The amount of money you have left in the bank
my_bank_value = 0.3

# The number of free transfers you have available for the upcoming gameweek
my_free_transfers = 1


# 2. RUN THE OPTIMIZER IN EVOLUTION MODE
print("\n" + "#"*60)
print("### RUNNING OPTIMIZER IN EVOLUTION MODE ###")
print("#"*60 + "\n")

# We pass our squad, bank, and FTs to the function.
total_score, initial_squad, plan = optimize_transfers_for_horizon(
    player_df=master_fpl_df,
    current_squad=my_team_squad,
    bank=my_bank_value,
    free_transfers_available=my_free_transfers,
)



############################################################
### RUNNING OPTIMIZER IN EVOLUTION MODE ###
############################################################

Solver is running... This might take a few moments.

Optimal strategy found! Total Expected Value over 6 weeks: 338.90

--- SQUAD FOR GAMEWEEK 1 ---
Total Squad Value: £100.4m | Money Remaining: £0.7m
--- Starting XI ---
      Name Role           Team Position  Price
      Raya             Arsenal       GK    5.5
    Virgil           Liverpool      DEF    6.1
     Munoz      Crystal Palace      DEF    5.6
    Senesi         Bournemouth      DEF    4.6
   M.Salah  (C)      Liverpool      MID   14.5
   Semenyo         Bournemouth      MID    7.5
  Grealish             Everton      MID    6.8
       Eze             Arsenal      MID    7.5
 L.Paqueta            West Ham      MID    5.9
  Gyokeres             Arsenal      FWD    9.0
Joao Pedro             Chelsea      FWD    7.7

--- Bench ---
      Name      Team Position  P